In [7]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import cv2
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
df_datos = pd.read_csv('data_imagenes.csv')
df_filtrado = df_datos[df_datos["label"] != 4.0]

print("Datos filtrados correctamente. Nuevas clases:", df_filtrado["label"].unique())
image_paths = df_datos['image_path'].values
labels = df_datos['label'].values

Datos filtrados correctamente. Nuevas clases: [0. 1. 2. 3.]


In [9]:
IMG_SIZE = (224, 224)
NUM_CLASSES = 4

df_train, df_test = train_test_split(df_filtrado, test_size=0.2, stratify=df_filtrado['label'], random_state=42)

In [10]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,       
    horizontal_flip=True,       
    fill_mode='nearest'     
)

test_datagen = ImageDataGenerator(rescale=1./255)

In [11]:
df_train['label'] = df_train['label'].astype(str)
df_test['label'] = df_test['label'].astype(str)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    x_col='image_path',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=8,
    class_mode='categorical',
    color_mode='grayscale',
    shuffle=True
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    x_col='image_path',
    y_col='label',
    target_size=IMG_SIZE,
    batch_size=16,
    class_mode='categorical',
    color_mode='grayscale',
    shuffle=False
)

Found 12521 validated image filenames belonging to 4 classes.
Found 3131 validated image filenames belonging to 4 classes.


In [14]:
def create_model():
    model = models.Sequential([
        layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 1)),
        
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        layers.Flatten(),
        
        layers.Dense(NUM_CLASSES, activation='softmax')
    ])
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), 
                  loss='categorical_crossentropy', 
                  metrics=['accuracy'])
    
    return model

In [15]:
model = create_model()

early_stopping = EarlyStopping(monitor='val_accuracy',
                               patience=5,
                               verbose=1,
                               mode='max',
                               baseline=0.97,
                               restore_best_weights=True)

model.fit(train_generator, epochs=15, validation_data=test_generator, callbacks=[early_stopping])

Epoch 1/15
1566/1566 ━━━━━━━━━━━━━━━━━━━━ 5032s 3s/step - accuracy: 0.5445 - loss: 1.9682 - val_accuracy: 0.6589 - val_loss: 0.8433
Epoch 2/15
1566/1566 ━━━━━━━━━━━━━━━━━━━━ 4825s 3s/step - accuracy: 0.6807 - loss: 0.8297 - val_accuracy: 0.6717 - val_loss: 0.8008
Epoch 3/15
1566/1566 ━━━━━━━━━━━━━━━━━━━━ 4856s 3s/step - accuracy: 0.7375 - loss: 0.6971 - val_accuracy: 0.7985 - val_loss: 0.5684
Epoch 4/15
1566/1566 ━━━━━━━━━━━━━━━━━━━━ 4907s 3s/step - accuracy: 0.7640 - loss: 0.6426 - val_accuracy: 0.8068 - val_loss: 0.5211
Epoch 5/15
1566/1566 ━━━━━━━━━━━━━━━━━━━━ 4864s 3s/step - accuracy: 0.7709 - loss: 0.6177 - val_accuracy: 0.8208 - val_loss: 0.4851
Epoch 6/15
1566/1566 ━━━━━━━━━━━━━━━━━━━━ 4903s 3s/step - accuracy: 0.7810 - loss: 0.5912 - val_accuracy: 0.8438 - val_loss: 0.4683
Epoch 7/15
1566/1566 ━━━━━━━━━━━━━━━━━━━━ 14477s 9s/step - accuracy: 0.8039 - loss: 0.5562 - val_accuracy: 0.8473 - val_loss: 0.4756
Epoch 8/15
1566/1566 ━━━━━━━━━━━━━━━━━━━━ 4979s 3s/step - accuracy: 0.8072 

In [16]:
model.save("ecg_cnn.keras")